In [4]:
#pip install python-okx --upgrade

In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import random
import matplotlib.pyplot as plt
import seaborn as sns 
import datetime
import math
import gc
import time


from IPython.display import clear_output

from sklearn.metrics import roc_auc_score

import lightgbm as lgbm

import pickle
def dump_pkl(data, filename):
  with open(filename, 'wb') as handle:
    pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)

def load_pkl(filename):
  with open(filename, 'rb') as handle:
    data = pickle.load(handle)
  return data



In [8]:
# apikey = "c326b1b4-58f5-46a2-abf0-0b0489d3165c"
# secretkey = "0D92FD5D337968108B492AE057B7D5D8"
# passphrase = 'PrimeTime27#'


### 1. Load data

In [3]:
import okx.MarketData as MarketData


def get_all_candles(ticker=None, start=None, end=None, interval=None):

    start_utc = (start - pd.Timedelta(hours=3)).value // 1000000  #3hour-utc
    end_utc = (end - pd.Timedelta(hours=3)).value // 1000000  #3hour-utc
    
    flag = "0"  # Production trading:0 , demo trading:1
    marketDataAPI =  MarketData.MarketAPI(flag=flag)

    times = []
    closes = []
    volumes = []
    
    cur_time_utc = end_utc
    while cur_time_utc > start_utc:
        try:
            clear_output()
            print(ticker, pd.to_datetime([cur_time_utc], unit='ms') + pd.Timedelta(hours=3))

            
            response = marketDataAPI.get_history_candlesticks(ticker, bar=interval, after=cur_time_utc, limit=100) 
            if response['code'] == '0':
                for elem in response['data']:
                        times += [elem[0]]
                        closes += [elem[4]]
                        volumes += [elem[5]]
                cur_time_utc = int(times[-1])

            if (response['code'] == '0') and (len(response['data']) == 0): #акции не так давно запущенные
                break 
            if response['code'] != '0':
                time.sleep(1)
                
        except:
            time.sleep(1)
            

    df = pd.DataFrame({'time' : times,
                      'close' : closes,
                      'volume' : volumes,
                        })
    
    df['time'] = pd.to_datetime(df['time'].astype(int), unit='ms') + pd.Timedelta(hours=3)
    df['close'] = df['close'].astype(np.float64)
    df['volume'] = df['volume'].astype(np.float64)
    df['ticker'] = ticker

    df = df[df['time'] >= start]
    df = df.sort_values(by='time').reset_index(drop=True)

    return df

    

In [15]:
%%time
df = get_all_candles(ticker='BTC-USDT', 
                start=pd.Timestamp('2025-01-01'),
                end=pd.Timestamp('2025-02-22'),
                interval='1H') #5m, 1H, 1Dutc
df

BTC-USDT DatetimeIndex(['2025-01-03'], dtype='datetime64[ns]', freq=None)
CPU times: user 132 ms, sys: 34.4 ms, total: 166 ms
Wall time: 4.32 s


,time,close,volume,ticker
0,2025-01-01 00:00:00,93892.7,176.749908,BTC-USDT
1,2025-01-01 01:00:00,93484.0,82.085040,BTC-USDT
2,2025-01-01 02:00:00,93585.1,125.375445,BTC-USDT
3,2025-01-01 03:00:00,94399.9,322.777877,BTC-USDT
4,2025-01-01 04:00:00,93604.7,186.278996,BTC-USDT
...,...,...,...,...
1243,2025-02-21 19:00:00,98227.9,809.763722,BTC-USDT
1244,2025-02-21 20:00:00,96974.0,957.709901,BTC-USDT
1245,2025-02-21 21:00:00,96528.1,1302.196432,BTC-USDT
1246,2025-02-21 22:00:00,95272.4,846.440868,BTC-USDT


In [32]:
stocks = [
    #'BTC-USDT',
#          'ETH-USDT',
#          'ETC-USDT',
#          'XRP-USDT',
#          'BNB-USDT',
#          'SOL-USDT',
#          'DOGE-USDT',
#          'ADA-USDT',
#          'TRX-USDT',
#          'LINK-USDT', #10
#          'AVAX-USDT', 
#          'SUI-USDT',
#          'XLM-USDT',
#          'LTC-USDT',
         'TON-USDT',
         # 'SHIB-USDT',
         # 'HBAR-USDT',
         # 'DOT-USDT',
         # 'BCH-USDT',
         # 'OP-USDT', #20
         'NEAR-USDT',
         # 'AAVE-USDT',
         # 'LDO-USDT',
         # 'ARB-USDT',
         # 'UNI-USDT',
         # 'ATOM-USDT', #26
         ]

start = pd.Timestamp("2024-01-01")
end = pd.Timestamp("2025-03-25") # не включительно

#
len(stocks)

2

### create dirs

In [36]:
!mkdir data

mkdir: data: File exists


In [38]:
!mkdir data/raw

mkdir: data/raw: File exists


In [40]:
!mkdir data/raw/1day data/raw/1hour data/raw/5min

mkdir: data/raw/1day: File exists
mkdir: data/raw/1hour: File exists
mkdir: data/raw/5min: File exists


#### #1day

In [44]:
#1day
for stock in tqdm(stocks):
    data = get_all_candles(ticker=stock,
                               start=start,
                               end=end,
                               interval='1Dutc') #5m, 1H, 1Dutc

    dump_pkl(data, f"./data/raw/1day/{stock}.pkl")

NEAR-USDT DatetimeIndex(['2024-02-19 03:00:00'], dtype='datetime64[ns]', freq=None)


100%|███████████████████████████████████████████████████████████████████████████| 2/2 [00:03<00:00,  1.72s/it]


#### #1hour

In [48]:
# stocks = ['TON-USDT',
#          'NEAR-USDT',]

# start = pd.Timestamp("2022-07-01")
# end = pd.Timestamp("2025-03-04") # не включительно

# #1hour



# for stock in tqdm(stocks):
#     data = get_all_candles(ticker=stock,
#                                start=start,
#                                end=end,
#                                interval='1H') #5m, 1H, 1Dutc

#     dump_pkl(data, f"./data/raw/1hour/{stock}.pkl")

In [57]:
#1hour



for stock in tqdm(stocks):
    data = get_all_candles(ticker=stock,
                               start=start,
                               end=end,
                               interval='1H') #5m, 1H, 1Dutc

    dump_pkl(data, f"./data/raw/1hour/{stock}.pkl")

NEAR-USDT DatetimeIndex(['2024-01-04 04:00:00'], dtype='datetime64[ns]', freq=None)


100%|███████████████████████████████████████████████████████████████████████████| 2/2 [01:09<00:00, 35.00s/it]


#### #5min

In [62]:
#5min
for stock in tqdm(stocks):
    data = get_all_candles(ticker=stock,
                               start=start,
                               end=end,
                               interval='5m') #5m, 1H, 1Dutc

    dump_pkl(data, f"./data/raw/5min/{stock}.pkl")

NEAR-USDT DatetimeIndex(['2024-01-01 01:00:00'], dtype='datetime64[ns]', freq=None)


100%|██████████████████████████████████████████████████████████████████████████| 2/2 [14:07<00:00, 423.62s/it]


### 2. Data Period

#### #1day

In [81]:
data_min = []
data_max = []
for stock in tqdm(stocks):
    df = load_pkl(f"./data/raw/1day/{stock}.pkl")
    data_min += [df['time'].min()]
    data_max += [df['time'].max()]

pd.DataFrame({'ticker' : stocks,
             'data_min' : data_min,
             'data_max' : data_max,
             })

100%|██████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 506.07it/s]


,ticker,data_min,data_max
0,TON-USDT,2024-01-01 03:00:00,2025-03-24 03:00:00
1,NEAR-USDT,2024-01-01 03:00:00,2025-03-24 03:00:00


#### #1hour

In [86]:
data_min = []
data_max = []
for stock in tqdm(stocks):
    df = load_pkl(f"./data/raw/1hour/{stock}.pkl")
    data_min += [df['time'].min()]
    data_max += [df['time'].max()]

pd.DataFrame({'ticker' : stocks,
             'data_min' : data_min,
             'data_max' : data_max,
             })

100%|██████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 285.79it/s]


,ticker,data_min,data_max
0,TON-USDT,2024-01-01,2025-03-24 23:00:00
1,NEAR-USDT,2024-01-01,2025-03-24 23:00:00


#### #5min

In [91]:
data_min = []
data_max = []
for stock in tqdm(stocks):
    df = load_pkl(f"./data/raw/5min/{stock}.pkl")
    data_min += [df['time'].min()]
    data_max += [df['time'].max()]

pd.DataFrame({'ticker' : stocks,
             'data_min' : data_min,
             'data_max' : data_max,
             })

100%|██████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 105.07it/s]


,ticker,data_min,data_max
0,TON-USDT,2024-01-01,2025-03-24 23:55:00
1,NEAR-USDT,2024-01-01,2025-03-24 23:55:00


### 3. Отсортированы, дубли, NaNs, дельты

#### #1day

In [116]:
# dfs = []
delta_time = pd.Timedelta(hours=24)

for stock in tqdm(stocks):
    df = load_pkl(f"./data/raw/1day/{stock}.pkl")
    
    if not (df['time'].sort_values()  == df['time']).all():
        print(f'{stock} not sorted: {(df['time'].sort_values()  != df['time']).sum()}')
        
    if df['time'].duplicated().any():
        print(f'{stock} duplicated: {df['time'].duplicated().sum()}')
        
    if not df.notnull().all().all():
        print(f'{stock} nulls: {df.isnull().sum()}')


    df['diff'] = df['time'].diff() / delta_time
    #проверка кратности delta_time
    if not (df['diff'].iloc[1:] % 1 == 0).all():
        print(f'{stock} not a multiple of delta time: {(df['diff'].iloc[1:] % 1 != 0).sum()}')

    #проверка пропущенных delta_time
    if not (df['diff'].iloc[1:] == 1).all():
        print(f'{stock} time gaps: {(df['diff'].iloc[1:] != 1).sum()}')
    
    
    # dfs += [df.copy()]


100%|████████████████████████████████████████████████████████████████████████| 26/26 [00:00<00:00, 619.85it/s]


In [118]:
#Время отсортировано, нет дублей по времени, нет NaNs в данных, временная сетка имеет кратность delta_time и нет пропусков совсем 

#### #1hour

In [127]:
# dfs = []
delta_time = pd.Timedelta(hours=1)

for stock in tqdm(stocks):
    df = load_pkl(f"./data/raw/1hour/{stock}.pkl")
    
    if not (df['time'].sort_values()  == df['time']).all():
        print(f'{stock} not sorted: {(df['time'].sort_values()  != df['time']).sum()}')
        
    if df['time'].duplicated().any():
        print(f'{stock} duplicated: {df['time'].duplicated().sum()}')
        
    if not df.notnull().all().all():
        print(f'{stock} nulls: {df.isnull().sum()}')


    df['diff'] = df['time'].diff() / delta_time
    #проверка кратности delta_time
    if not (df['diff'].iloc[1:] % 1 == 0).all():
        print(f'{stock} not a multiple of delta time: {(df['diff'].iloc[1:] % 1 != 0).sum()}')

    #проверка пропущенных delta_time
    if not (df['diff'].iloc[1:] == 1).all():
        print(f'{stock} time gaps: {(df['diff'].iloc[1:] != 1).sum()}')
    
    
    # dfs += [df.copy()]


100%|████████████████████████████████████████████████████████████████████████| 26/26 [00:00<00:00, 379.65it/s]


In [129]:
#Время отсортировано, нет дублей по времени, нет NaNs в данных, временная сетка имеет кратность delta_time и нет пропусков совсем 

#### #5min

In [133]:
# dfs = []
delta_time = pd.Timedelta(minutes=5)

for stock in tqdm(stocks):
    df = load_pkl(f"./data/raw/5min/{stock}.pkl")
    
    if not (df['time'].sort_values()  == df['time']).all():
        print(f'{stock} not sorted: {(df['time'].sort_values()  != df['time']).sum()}')
        
    if df['time'].duplicated().any():
        print(f'{stock} duplicated: {df['time'].duplicated().sum()}')
        
    if not df.notnull().all().all():
        print(f'{stock} nulls: {df.isnull().sum()}')


    df['diff'] = df['time'].diff() / delta_time
    #проверка кратности delta_time
    if not (df['diff'].iloc[1:] % 1 == 0).all():
        print(f'{stock} not a multiple of delta time: {(df['diff'].iloc[1:] % 1 != 0).sum()}')

    #проверка пропущенных delta_time
    if not (df['diff'].iloc[1:] == 1).all():
        print(f'{stock} time gaps: {(df['diff'].iloc[1:] != 1).sum()}')
    
    
    # dfs += [df.copy()]


100%|█████████████████████████████████████████████████████████████████████████| 26/26 [00:00<00:00, 51.48it/s]


In [135]:
#Время отсортировано, нет дублей по времени, нет NaNs в данных, временная сетка имеет кратность delta_time и нет пропусков совсем 

In [140]:
#Как-то через чур хорошо, что даже разрывов во времени нет, какая-то сказка

### 4. Preproc (склейка с BTC)

In [148]:
stocks = [#'BTC-USDT',
         'ETH-USDT',
         'ETC-USDT',
         'XRP-USDT',
         'BNB-USDT',
         'SOL-USDT',
         'DOGE-USDT',
         'ADA-USDT',
         'TRX-USDT',
         'LINK-USDT', 
         'AVAX-USDT', 
         'SUI-USDT',
         'XLM-USDT',
         'LTC-USDT',
         'TON-USDT',
         'SHIB-USDT',
         'HBAR-USDT',
         'DOT-USDT',
         'BCH-USDT',
         'OP-USDT', 
         'NEAR-USDT',
         'AAVE-USDT',
         'LDO-USDT',
         'ARB-USDT',
         'UNI-USDT',
         'ATOM-USDT', 
         ]
len(stocks)

25

In [150]:
!mkdir ./data/preproc

In [152]:
!mkdir ./data/preproc/1day ./data/preproc/1hour ./data/preproc/5min

#### #1day

In [163]:
data_btc = load_pkl(f"./data/raw/1day/BTC-USDT.pkl")

for stock in tqdm(stocks):
    data = load_pkl(f"./data/raw/1day/{stock}.pkl")

    data_ext = data.merge(data_btc[['time', 'close']].copy().rename(columns={"close" : "btc_close"}), how='left', on='time')
    assert data.shape[0] == data_ext.shape[0], 'Error 1'
    
    assert data_ext['btc_close'].isnull().sum() == 0, 'Error 2'
    #data['btc_close'] = data['btc_close'].ffill()
    
    dump_pkl(data_ext, f"./data/preproc/1day/{stock}.pkl")

100%|████████████████████████████████████████████████████████████████████████| 25/25 [00:00<00:00, 487.56it/s]


#### #1hour

In [167]:
data_btc = load_pkl(f"./data/raw/1hour/BTC-USDT.pkl")

for stock in tqdm(stocks):
    data = load_pkl(f"./data/raw/1hour/{stock}.pkl")

    data_ext = data.merge(data_btc[['time', 'close']].copy().rename(columns={"close" : "btc_close"}), how='left', on='time')
    assert data.shape[0] == data_ext.shape[0], 'Error 1'
    
    assert data_ext['btc_close'].isnull().sum() == 0, 'Error 2'
    #data['btc_close'] = data['btc_close'].ffill()
    
    dump_pkl(data_ext, f"./data/preproc/1hour/{stock}.pkl")

100%|████████████████████████████████████████████████████████████████████████| 25/25 [00:00<00:00, 256.66it/s]


#### #5min

In [171]:
data_btc = load_pkl(f"./data/raw/5min/BTC-USDT.pkl")

for stock in tqdm(stocks):
    data = load_pkl(f"./data/raw/5min/{stock}.pkl")

    data_ext = data.merge(data_btc[['time', 'close']].copy().rename(columns={"close" : "btc_close"}), how='left', on='time')
    assert data.shape[0] == data_ext.shape[0], 'Error 1'
    
    assert data_ext['btc_close'].isnull().sum() == 0, 'Error 2'
    #data['btc_close'] = data['btc_close'].ffill()
    
    dump_pkl(data_ext, f"./data/preproc/5min/{stock}.pkl")

100%|█████████████████████████████████████████████████████████████████████████| 25/25 [00:00<00:00, 55.97it/s]


### -1. Check data

In [ ]:
#чекнуть сами данные и совпадение с индексом

#### #1day

In [213]:
i = 900
ticker = 'TON-USDT'

df = load_pkl(f"./data/preproc/1day/{ticker}.pkl")
df.iloc[i-1:i+1]

,time,close,volume,ticker,btc_close
899,2024-12-16 03:00:00,6.059,7.826239e+06,TON-USDT,106066.0
900,2024-12-17 03:00:00,5.822,7.610322e+06,TON-USDT,106128.4


In [242]:
i = 890
ticker = 'SHIB-USDT'

df = load_pkl(f"./data/preproc/1day/{ticker}.pkl")
df.iloc[i-1:i+1]

,time,close,volume,ticker,btc_close
889,2024-12-06 03:00:00,0.000031,1.638883e+12,SHIB-USDT,99734.0
890,2024-12-07 03:00:00,0.000032,2.286781e+12,SHIB-USDT,99835.4


In [224]:
i = 750
ticker = 'ETH-USDT'

df = load_pkl(f"./data/preproc/1day/{ticker}.pkl")
df.iloc[i-1:i+1]

,time,close,volume,ticker,btc_close
749,2024-07-19 03:00:00,3504.20,80959.220153,ETH-USDT,66659.9
750,2024-07-20 03:00:00,3517.35,45096.645280,ETH-USDT,67137.7


#### #1hour

In [277]:
i = 23000
ticker = 'NEAR-USDT'

df = load_pkl(f"./data/preproc/1hour/{ticker}.pkl")
df.iloc[i-1:i+1]

,time,close,volume,ticker,btc_close
22999,2025-02-13 07:00:00,3.386,26114.925675,NEAR-USDT,96949.0
23000,2025-02-13 08:00:00,3.375,21805.760994,NEAR-USDT,96669.9


In [271]:
i = 21000
ticker = 'SHIB-USDT'

df = load_pkl(f"./data/preproc/1hour/{ticker}.pkl")
df.iloc[i-1:i+1]

,time,close,volume,ticker,btc_close
20999,2024-11-21 23:00:00,0.000025,3.435063e+10,SHIB-USDT,98107.9
21000,2024-11-22 00:00:00,0.000025,2.848376e+10,SHIB-USDT,97967.2


In [281]:
i = 17000
ticker = 'BNB-USDT'

df = load_pkl(f"./data/preproc/1hour/{ticker}.pkl")
df.iloc[i-1:i+1]

,time,close,volume,ticker,btc_close
16999,2024-11-28 13:00:00,658.4,1603.791742,BNB-USDT,94952.4
17000,2024-11-28 14:00:00,659.9,1427.955300,BNB-USDT,95456.0


#### #5min

In [330]:
i = -202
ticker = 'DOGE-USDT'

df = load_pkl(f"./data/preproc/5min/{ticker}.pkl")
df.iloc[i-2:i+1]

,time,close,volume,ticker,btc_close
278868,2025-02-23 07:00:00,0.24345,6.913738e+05,DOGE-USDT,96388.1
278869,2025-02-23 07:05:00,0.24323,1.477939e+06,DOGE-USDT,96398.1
278870,2025-02-23 07:10:00,0.24351,7.856233e+05,DOGE-USDT,96448.1


In [332]:
df['volume'].iloc[i-2:i+1].sum()

2954936.2011720003

In [334]:
i = -5002
ticker = 'SHIB-USDT'

df = load_pkl(f"./data/preproc/5min/{ticker}.pkl")
df.iloc[i-2:i+1]

,time,close,volume,ticker,btc_close
274068,2025-02-06 15:00:00,0.000016,1.705860e+09,SHIB-USDT,98982.9
274069,2025-02-06 15:05:00,0.000016,2.923923e+09,SHIB-USDT,98642.1
274070,2025-02-06 15:10:00,0.000016,1.175243e+09,SHIB-USDT,98604.4


In [336]:
df['volume'].iloc[i-2:i+1].sum()

5805025017.0

In [338]:
i = -1000
ticker = 'SOL-USDT'

df = load_pkl(f"./data/preproc/5min/{ticker}.pkl")
df.iloc[i-2:i+1]

,time,close,volume,ticker,btc_close
278070,2025-02-20 12:30:00,172.28,775.086874,SOL-USDT,97191.1
278071,2025-02-20 12:35:00,172.12,583.303418,SOL-USDT,97190.1
278072,2025-02-20 12:40:00,172.35,1796.954872,SOL-USDT,97276.0


In [340]:
df['volume'].iloc[i-2:i+1].sum()

3155.3451640000003

In [ ]:
#Все бьется с терминальными данными =)

In [3]:
279080+140300+141200 -571197

-10617